##Setup libraries, dataset path and movie genres

In [48]:
import pandas as pd
import os
dataset_path = "../dataset"
movie_genres = ['action', 'adventure', 'animation', 'biography', 'crime', 'family', 'fantasy', 
          'film-noir', 'history', 'horror', 'mystery', 'romance', 'scifi', 'sports', 
          'thriller', 'war']

1. Read each file and store dataframes separately

In [50]:
# Define a dictionary to store dataframes for each genre
movies_by_genre = {}

# Update the dictionary
for file in os.listdir(dataset_path):
    for genre in movie_genres:
        if genre in file:
            if genre not in movies_by_genre:
                movies_by_genre[genre] = pd.read_csv(os.path.join(dataset_path, file))
            else:
                movies_by_genre[genre] = pd.concat([movies_by_genre[genre], pd.read_csv(os.path.join(dataset_path, file))])

# Print the number of rows and columns in each genre dataframe without any data quality checks
print("genre, number of rows, number of columns")
for genre, movies_df in movies_by_genre.items():
    print(genre, movies_df.shape)

genre, number of rows, number of columns
action (52452, 14)
adventure (25664, 14)
animation (8419, 14)
biography (8289, 14)
crime (35852, 14)
family (17095, 14)
fantasy (17163, 14)
film-noir (986, 14)
history (8996, 14)
horror (36682, 14)
mystery (18960, 14)
romance (52617, 14)
scifi (16557, 14)
sports (5292, 14)
thriller (53365, 14)
war (9911, 14)


2. Perform data quality checks

In [72]:
#  for each dataframe in the dictionary, rename the columns and select only the columns we need
for genre, movies_df in movies_by_genre.items():
    movies_df = movies_df.rename(columns={'gross(in $)': 'gross'})
    movies_df = movies_df.rename(columns={'movie_name': 'movie'})
    movies_df = movies_df[['movie', 'year', 'runtime', 'rating', 'director', 'star', 'votes', 'gross']]
    # print the columns selected for each genre
    print(genre, movies_df.columns.values)
    
    

action ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
adventure ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
animation ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
biography ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
crime ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
family ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
fantasy ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
film-noir ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
history ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
horror ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
mystery ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
romance ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'votes' 'gross']
scifi ['movie' 'year' 'runtime' 'rating' 'director' 'star' 'v